# Artificial Neural Networks And Deep Learning 
# Homework 1: Image Classification
# CNN Model from Scratch
Students: 

*   Julián Jiménez: 10657117
*   Samuel Polo: 10670388

        

# **Our approach of the Task:**
# Initial approach

We started the project with the basic CNN Model that we created in the Laboratory session of the class in order to do some basic tests. Without making any changes, our first score was 0.49800. 

After this, we began to do some minor changes in order to increase our perfomance, and our second test made was making data augmentation, in which we increased our score up to 0.61600.

Then we settled the following list of tests:



*   The same model with L2 regularization (score: 0.57399)
*   Modification of the split value to 0.25 and L1_L2 regularization (score: 0.55600)

*   Additions of neurons to the fully connected layer (score: 0.51800)
*   Additions of neurons and regularization (score: 0.53200)

*   Modification of Elu as activation function with the previous model (score: 0.53400)
*   The previous model with a modified validation set separation (score: 0.57799)

*   The previous model with a greater number of filters for the CNN (score: 0.51800)

Surprisingly, after all our tests we couldn't improve the result from our second test, so we decided to approach the problem with the Transfer Learning method.




In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

2.0.0


In [0]:
import numpy as np
import os

In [0]:
# Set the seed for random operations. 
# This lets our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  
np.random.seed(SEED)

# Get current working directory
cwd = os.getcwd()
print(cwd)

/content


In [0]:
#Set the directories for all, training and validation images.
dataset_dir = os.path.join(cwd, 'Classification_Dataset')
training_dir = os.path.join(dataset_dir,'training')
valid_dir = os.path.join(dataset_dir,'validation')

## Validation Directory Creation:

This code is used to create the validation directory and fill it up with images from all the classes.
The validation set is created with stratified sampling since all classes are not represented equally in the given dataset. In this way from each class we select the same proportion of images. This is done to reduce sampling error and allow all classes to be represented in the validation set.

In [0]:
#Creates validation directory if it does not exist already.
_ = os.makedirs(valid_dir, exist_ok=True)

In [0]:
#Proportion for Hold-out set validation.
train_valid_split = 0.2

#Fills the validtaion set 
subfolders = [f.path for f in os.scandir(training_dir) if f.is_dir() ] 

for subfold in subfolders:
  head_tail = os.path.split(subfold)
  _ = os.makedirs(os.path.join(valid_dir,head_tail[1]), exist_ok=True)
  list_files = [name for name in os.scandir(subfold) if os.path.isfile(name)]
  num_files = len(list_files)
  for num in np.random.choice(num_files, int(num_files*0.2) ,replace=False):
    file_name = os.path.split(list_files[num])
    os.rename(list_files[num],os.path.join(valid_dir,head_tail[1],file_name[1]))

# Training and Validation Generator and Datasets

In [0]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=20,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)
    
valid_data_gen = ImageDataGenerator(rescale=1./255)
    

In [0]:
#Batch Size
bs = 64
# img shape
img_h = 256
img_w = 256

num_classes = 20

classes = ['owl',              # 0
           'galaxy',           # 1
           'lightning',        # 2
           'wine-bottle',      # 3
           't-shirt',          # 4
           'waterfall',        # 5
           'sword',            # 6
           'school-bus',       # 7
           'calculator',       # 8
           'sheet-music',      # 9
           'airplanes',        # 10
           'lightbulb',        # 11
           'skyscraper',       # 12
           'mountain-bike',     # 13
           'fireworks',        # 14
           'computer-monitor', # 15
           'bear',             # 16
           'grand-piano',      # 17
           'kangaroo',         # 18
           'laptop']           # 19

In [0]:
# Training Generator
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               target_size=(img_h,img_w),
                                               seed=SEED)

Found 1247 images belonging to 20 classes.


In [0]:
#Validation Generator
valid_gen = train_data_gen.flow_from_directory(valid_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=False,
                                               target_size=(img_h,img_w),
                                               seed=SEED)

Found 307 images belonging to 20 classes.


In [0]:
#Create Dataset objects
#Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()

In [0]:
#Validation
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
valid_dataset = valid_dataset.repeat()

# **Creation of the CNN Model**
## **Keras Model Subclass**

In [0]:
# Keras Model subclassing 

# Create convolutional block
class ConvBlock(tf.keras.Model):
    def __init__(self, num_filters):
        super(ConvBlock, self).__init__()
        self.conv2d = tf.keras.layers.Conv2D(filters=num_filters,
                                             kernel_size=(3, 3),
                                             strides=(1, 1), 
                                             padding='same')
        self.activation = tf.keras.layers.ReLU() 
        self.pooling = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
        
    def call(self, inputs):
        x = self.conv2d(inputs)
        x = self.activation(x)
        x = self.pooling(x)
        return x

In [0]:
# Create Model
# ------------

depth = 5
start_f = 64
num_classes = 20

class CNNClassifier(tf.keras.Model):
    def __init__(self, depth, start_f, num_classes):
        super(CNNClassifier, self).__init__()
        
        self.feature_extractor = tf.keras.Sequential()
    
        for i in range(depth):
            self.feature_extractor.add(ConvBlock(num_filters=start_f))
            start_f *= 2
            
        self.flatten = tf.keras.layers.Flatten()
        self.classifier = tf.keras.Sequential()
        self.classifier.add(tf.keras.layers.Dense(units=4096, activation='relu'))
        self.classifier.add(tf.keras.layers.Dense(units=4096, activation='relu'
                                                  ))
        self.classifier.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
        
    def call(self, inputs):
        x = self.feature_extractor(inputs)
        x = self.flatten(x)
        x = self.classifier(x)
        return x
    
# Create Model instance
model = CNNClassifier(depth=depth,
                      start_f=start_f,
                      num_classes=num_classes)
# Build Model (Required)
model.build(input_shape=(None, img_h, img_w, 3))

## **Definition of Parameters for Training**

In [0]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [0]:
# Visualize created model as a table
model.feature_extractor.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_block_10 (ConvBlock)    multiple                  1792      
_________________________________________________________________
conv_block_11 (ConvBlock)    multiple                  73856     
_________________________________________________________________
conv_block_12 (ConvBlock)    multiple                  295168    
_________________________________________________________________
conv_block_13 (ConvBlock)    multiple                  1180160   
_________________________________________________________________
conv_block_14 (ConvBlock)    multiple                  4719616   
Total params: 6,270,592
Trainable params: 6,270,592
Non-trainable params: 0
_________________________________________________________________


# **Training Procedure**

In [0]:
from datetime import datetime

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Early Stopping
# Restores the best weights found during the training
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    callbacks.append(es_callback)


model.fit(x=train_dataset,
          epochs=200,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Train for 20 steps, validate for 5 steps
Epoch 1/200
20/20 [==============================] - 57s 3s/step - loss: 4.5232 - accuracy: 0.0569 - val_loss: 2.9659 - val_accuracy: 0.0651
Epoch 2/200
20/20 [==============================] - 57s 3s/step - loss: 2.9738 - accuracy: 0.0666 - val_loss: 2.9271 - val_accuracy: 0.1205
Epoch 3/200
20/20 [==============================] - 57s 3s/step - loss: 2.8836 - accuracy: 0.0962 - val_loss: 2.8022 - val_accuracy: 0.1173
Epoch 4/200
20/20 [==============================] - 57s 3s/step - loss: 2.7775 - accuracy: 0.0954 - val_loss: 2.8520 - val_accuracy: 0.1173
Epoch 5/200
20/20 [==============================] - 57s 3s/step - loss: 2.7861 - accuracy: 0.1115 - val_loss: 2.7165 - val_accuracy: 0.1336
Epoch 6/200
20/20 [==============================] - 56s 3s/step - loss: 2.7222 - accuracy: 0.1195 - val_loss: 2.7806 - val_accuracy: 0.0945
Epoch 7/200
20/20 [==============================] - 56s 3s/step - loss: 2.6907 - accuracy: 0.1235 - val_loss: 2.

# **Output of the Solution**

# **CSV Creation**

In [0]:
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

## **Test set label assignment**

In [0]:
from PIL import Image
from skimage import transform
test_dir = os.path.join(dataset_dir, 'test')

image_filenames = next(os.walk(test_dir))[2]

results = {}
for image_name in image_filenames:
  #print(image_name)
  image_dir = os.path.join(test_dir,image_name)
  img = Image.open(image_dir).convert('RGB')
  img_array = np.array(img)
  img_array = np.array(img_array).astype('float32')/255.
  img_array = transform.resize(img_array, (256, 256, 3))
  img_array = np.expand_dims(img_array, 0)
  pred = model.predict(img_array)
  prediction = np.argmax(model.predict(img_array))   # predicted class
  results[image_name] = prediction

create_csv(results)